<!--Información del curso-->
<img align="left" style="padding-right:10px;" src="figuras/logo_ciencia_datos.png">


<center><h2 style="font-size:2em;color:#840700">  Agrupando datos  </h4></center>

<br>
<table>
<col width="550">
<col width="450">
<tr>
<td><img src="figuras/agrupando.png" align="left" style="width:500px"/></td>
<td>

* **Wes McKinney**, empezó a desarrollar Pandas en el año 2008 mientras trabajaba en *AQR Capital* [https://www.aqr.com/] por la necesidad que tenía de una herramienta flexible de alto rendimiento para realizar análisis cuantitativos en datos financieros. 
* Antes de dejar AQR convenció a la administración de la empresa de distribuir esta biblioteca bajo licencia de código abierto.
* **Pandas** es un acrónimo de **PANel DAta analysiS**
   
    
<br>
</td>
</tr>
</table>

# Librerías

Cargando las bibliotecas que necesitamos 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Datos utilizados en esta Notebook

El archivo *planetas.csv* contiene datos sobre planetas que se han descubierto orbitando estrellas fuera de nuestro sistema solar. En este archivo, cada fila corresponde a un exoplaneta descubierto. Los atributos de cada exoplaneta (y por lo tanto las columnas del archivo) son:

* **método**  el método utilizado para descubrir el planeta.
* **número** el número total de planetas descubiertos orbitando la estrella anfitriona de este exoplaneta.
* **orbital_period** el período del planeta, su "año".
* **masa** la masa del exoplaneta.
* **distancia** la distancia de la estrella anfitriona del exoplaneta a la Tierra en años luz.
* **año** el año en que se descubrió el planeta.

In [2]:
# cargar el archivo datos/planets.csv en un dataframe llamado df_planetas


,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [3]:
#Numero total de filas y columnas


(1035, 6)

In [4]:
#Caracteristicas de cada columna


,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


Como se puede observar existen columnas que no tienen todos los datos (ver el parametro **count** que es diferente, solo cuenta cuando existe el dato)

#  Introducción

A menudo se require agrupar a través de alguna etiqueta en los *DataFrtames*: esto se implementa en la operación denominada ``groupby``. El nombre "group by" o "agrupar por"  proviene de un comando en el lenguaje de la base de datos **SQL**, pero quizás sea más esclarecedor pensar en los términos acuñados por primera vez por Hadley Wickham: dividir, aplicar, combinar.

En esta figura se ilustra un ejemplo canónico de esta operación *dividir-aplicar-combinar (split-apply-combine)*, donde "aplicar" es una *agregación de suma*:

<img align="left" width="550"  float= "none" align="middle" src="figuras/groupby.png">


Esto deja en claro lo que logra ``groupby``:

- El paso *split* o *dividir*, implica dividir y agrupar un *DataFrame* según la etiqueta especificada.
- El paso *apply* o *aplicar*, implica calcular alguna función, generalmente un agregado, transformación o filtrado, dentro de los grupos individuales.
- El paso  *combine* o *combinar*, fusiona los resultados de estas operaciones en un arreglo de salida.

Si bien esto ciertamente se podría hacer manualmente usando alguna combinación de los comandos de enmascaramiento, agregación y concatenación cubiertos en las lecciones anteriores, el poder de ``groupby`` es que abstrae estos pasos: el usuario no necesita pensar en *cómo* se realiza el cálculo, sino que piensa en la *operación como un todo*.

Como ejemplo concreto, echemos un vistazo al uso de **Pandas** para el cálculo que se muestra en este diagrama.
Comenzaremos creando el  *DataFrame*

In [5]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


La operación más básica de dividir-aplicar-combinar se puede calcular con el método ``groupby``, pasando el nombre de la columna de etiqueta deseada:

In [6]:
#groupby('key')


Observe que lo que se retorna no es un conjunto de *DataFrames*, sino un objeto ``DataFrameGroupBy``. Este objeto es donde está la magia: puede pensar en él como una vista especial del DataFrame, que está preparado para profundizar en los grupos, pero no realiza ningún cálculo real hasta que se aplica la agregación. Este enfoque de "evaluación perezosa" significa que los agregados comunes se pueden implementar de manera muy eficiente de una manera casi transparente para el usuario.

Para producir un resultado, podemos aplicar un agregado a este objeto ``DataFrameGroupBy``, que realizará los pasos apropiados de aplicación/combinación para producir el resultado deseado:

In [7]:
#Sumando los elementos de las etiquetas que coinciden
#groupby('key').sum()

,data
key,
A,3
B,5
C,7


El método ``sum()``  es solo una posibilidad aquí; puede aplicar virtualmente cualquier función de agregación Pandas o NumPy común, así como virtualmente cualquier operación válida de *DataFrame*

In [8]:
#El elemento mayor 
#groupby('key').max()

,data
key,
A,3
B,4
C,5


### El objeto GroupBy

El objeto ``GroupBy`` es una abstracción muy flexible. En muchos sentidos, puede simplemente tratarlo como si fuera una colección de *DataFrames* y hace las cosas difíciles sin que se percate de las operaciones y cálculos realizados. Veamos algunos ejemplos usando los datos de exoplanetas.

Quizás las operaciones más importantes que ofrece un ``GroupBy`` son el *agregado*, *filtrado*, *transformación* y *aplicación de un función*.

Discutiremos cada uno de estos con más detalle en las siguientes subsecciones, pero antes de eso, vamos a presentar algunas de las otras funciones que se pueden usar con la operación básica de ``GroupBy``.

#### Indexación de columnas

El objeto ``GroupBy`` admite la indexación de columnas de la misma manera que el *DataFrame* y devuelve un objeto ``GroupBy`` modificado.
Por ejemplo:

In [9]:
#groupby('method') 


In [10]:
# method -> 'orbital_period' 


Aquí hemos seleccionado un grupo en particular del grupo *DataFrame* original por referencia a su nombre de columna. Al igual que con el objeto ``GroupBy``, no se realiza ningún cálculo hasta que llamamos a algún agregado en el objeto:

In [11]:
# method -> 'orbital_period' -> median()  
 

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

Si requerimos la lista ordenada alfabéticamente requerimos utilizar la función ``reset_index()``

In [12]:
#reset_index()

,method,orbital_period
0,Astrometry,631.180000
1,Eclipse Timing Variations,4343.500000
2,Imaging,27500.000000
3,Microlensing,3300.000000
4,Orbital Brightness Modulation,0.342887
5,Pulsar Timing,66.541900
6,Pulsation Timing Variations,1170.000000
7,Radial Velocity,360.200000
8,Transit,5.714932
9,Transit Timing Variations,57.011000


Ahora es posible aplicar el orden a la columna "orbital_period"

In [13]:
#ordenando


,method,orbital_period
2,Imaging,27500.000000
1,Eclipse Timing Variations,4343.500000
3,Microlensing,3300.000000
6,Pulsation Timing Variations,1170.000000
0,Astrometry,631.180000
7,Radial Velocity,360.200000
5,Pulsar Timing,66.541900
9,Transit Timing Variations,57.011000
8,Transit,5.714932
4,Orbital Brightness Modulation,0.342887


Esto da una idea de la escala general de períodos orbitales (en días) a los que es sensible cada método. Podemos hacer lo mismo agrupando con lo el año.

In [14]:
# 'year' -> 'distance' -> median() 


year
1989     40.570
1992        NaN
1994        NaN
1995     15.360
1996     14.840
1997     17.430
1998     21.290
1999     29.760
2000     32.865
2001     33.650
2002     37.440
2003     35.955
2004     48.950
2005     35.870
2006     46.940
2007     80.580
2008     56.070
2009     52.830
2010     90.000
2011     87.870
2012    200.000
2013    132.000
2014    272.000
Name: distance, dtype: float64

Como era de esperarse se descubren planetas mas lejanos conforme va mejorando la tecnología a través de los años.